In [ ]:
## prevent colab shutdown 
%%javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}

var connection = setInterval(ConnectButton, 60000);

function myStopFunction() {
    console.log("Remove interal"); 
    clearInterval(connection);
}

<IPython.core.display.Javascript object>

In [ ]:
%%capture
## install kaggle
!pip install kaggle

import os
import json
import glob
import shutil

## make sure gdrive is mounted
if not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount("/content/drive")

## pull repo
if not os.path.exists('/content/melanoma-classification'):
    !git clone https://github.com/SaschaMet/melanoma-classification.git

## update repo
!cd melanoma-classification && git fetch && git pull

In [10]:
## switch branch and execute function
!cd /content/melanoma-classification && git checkout "add-train-function" && git fetch && git pull

Already up to date.


In [ ]:
%%capture
## download dataset and move it to the right place
!python /content/melanoma-classification/src/download_data.py
!rm -rf /content/melanoma-classification/data
!mv /content/data /content/melanoma-classification/data

In [11]:
!python /content/melanoma-classification/src/train-model.py

2021-02-13 13:11:36.091358: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-02-13 13:11:38.240621: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-13 13:11:38.241511: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-02-13 13:11:38.277722: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-02-13 13:11:38.278403: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-02-13 13:11:38.278462: I tensorflow/stream_executor/platform/default/dso_loade

In [12]:
# remove the old evaluation
!rm -rf /content/melanoma-classification/evaluation

# create a new folder
!mkdir /content/melanoma-classification/evaluation

# copy the training files to the evaluation dir
os.chdir("/content")
for file in glob.glob("*.png"):
    fileName = file.split("-")[-1]
    shutil.move(file, "/content/melanoma-classification/evaluation/" + fileName)

for file in glob.glob("*.hdf5"):
    fileName = file.split("-")[-1]
    shutil.move(file, "/content/melanoma-classification/evaluation/" + fileName)

for file in glob.glob("*.json"):
    fileName = file.split("-")[-1]
    shutil.move(file, "/content/melanoma-classification/evaluation/" + fileName)

for file in glob.glob("*.txt"):
    shutil.move(file, "/content/melanoma-classification/evaluation/" + file)

In [ ]:
# push the files to github
!cd melanoma-classification && git status
!cd melanoma-classification && git remote set-url origin $(head -n 1 "/content/drive/MyDrive/Colab Notebooks/_auth/gh_auth.txt")
!cd melanoma-classification && git add . && git commit -m "colab training" && git push

In [ ]:
os.chdir("/content")
for file in glob.glob("*.png"):
    os.remove(file)

for file in glob.glob("*.hdf5"):
    os.remove(file)

for file in glob.glob("*.json"):
    os.remove(file)

for file in glob.glob("*.txt"):
    os.remove(file)